In [2]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets import DatasetFolder
import cv2
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [78]:
from torchvision import transforms
import PIL.Image

from sklearn.preprocessing import StandardScaler


labels = {0: "n02089078-black-and-tan_coonhound"
          ,1: "n02091831-Saluki"
          ,2:"n02092002-Scottish_deerhound"
          ,3:"n02095314-wire-haired_fox_terrier"}

def load_image(img_path:str):
    np_img = cv2.imread(img_path,cv2.IMREAD_COLOR)
    np_img = cv2.cvtColor(np_img,cv2.COLOR_BGR2GRAY)
    data = cv2.calcHist([np_img],[0], None, [256], [0, 256])
    return data
dset = DatasetFolder(root=r'../DataSet/ProcessedDatasets', loader = load_image,extensions = ('.jpg',))

data = StandardScaler().fit_transform(dset[0][:][0]) 



[-0.9708298]


In [ ]:
import torch.utils.data
from torch.utils.data import Sampler

train_loader = 